In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../python_tools/')
from tools import *
from tools_mapping import *

In [ ]:
filename='/data/ptiengou/PFTmap_2014.nc'
map0=xr.open_dataset(filename, decode_times=False)
#select subset
map=map0.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))
map

<xarray.Dataset>
Dimensions:       (lat: 0, lon: 140, time_counter: 1, veget: 15)
Coordinates:
  * lat           (lat) float32 
  * lon           (lon) float32 -9.95 -9.85 -9.75 -9.65 ... 3.65 3.75 3.85 3.95
  * time_counter  (time_counter) float32 0.0
  * veget         (veget) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
Data variables:
    maxvegetfrac  (time_counter, veget, lat, lon) float32 ...
Attributes: (12/19)
    description:  ORCHIDEE PFT map derived from ESACCI Land Cover map v2.0.7b...
    PFT1:         bare soil
    PFT2:         tropical broadleaf evergreen
    PFT3:         tropical broadleaf raingreen
    PFT4:         temperate needleleaf evergreen
    PFT5:         temperate broadleaf evergreen
    ...           ...
    PFT13:        C4 agriculture
    PFT14:        tropical C3 grass
    PFT15:        boreal C3 grass
    contact:      V.Bastrikov (vladislav.bastrikov@lsce.ipsl.fr), P.Peylin (p...
    web:          https://orchidas.lsce.ipsl.fr/dev/lccci/
    date:         27/12/2022 00:33

: 

In [ ]:
#Display map of var for ds on restricted area
var='maxvegetfrac'
nb=1
ds = map0
# ds = std
latmin=36.0
latmax=44
lonmin=-10
lonmax=3.5

#restrict ds to latmin, latmax, lonmin, lonmax
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)
plotvar = ds[var][:,:,:,nb]

map_plotvar(plotvar, cmap=myvir,title=var)